In [54]:
import config
import requests
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
api_url = "https://api.clashroyale.com/v1"
us_location_id = 57000249
headers = {
    "content-type": "application/json",
    "Authorization": f"Bearer:{config.token}"
}

In [3]:
def get_list_locations():
    response = requests.get(f"{api_url}/locations", headers=headers)
    json_response = response.json()
    return json_response

def get_list_locations_rankings_players(location_id):
    response = requests.get(f"{api_url}/locations/{location_id}/rankings/players", headers=headers)
    json_response = response.json()
    return json_response

def get_top_ranking_player_by_location(location_id):
    response = requests.get(f"{api_url}/locations/{location_id}/rankings/players", headers=headers)
    json_response = response.json()
    return json_response['items'][0]

def convert_player_tag_hash(player_tag):
    player_tag = player_tag.replace('#', '%23')
    return player_tag

def get_player_info(player_tag):
    player_tag = convert_player_tag_hash(player_tag)
    response = requests.get(f"{api_url}/players/{player_tag}", headers=headers)
    json_response = response.json()
    return json_response

def get_player_battlelog(player_tag):
    player_tag = convert_player_tag_hash(player_tag)
    response = requests.get(f"{api_url}/players/{player_tag}/battlelog", headers=headers)
    json_response = response.json()
    return json_response

def get_list_of_pvp_battles_from_player(player_tag):
    player_battlelog = get_player_battlelog(player_tag)
    list_of_pvp_battles = []
    for battle_type in player_battlelog:
        if battle_type['type'] == 'PvP':
            list_of_pvp_battles.append(battle_type)
    return list_of_pvp_battles


In [4]:
us_top_ranking_player = get_top_ranking_player_by_location(us_location_id)

In [5]:
player_battlelog = get_player_battlelog(us_top_ranking_player['tag'])

In [6]:
player_pvp_battlelog = get_list_of_pvp_battles_from_player(us_top_ranking_player['tag'])

In [7]:
player_pvp_battlelog[0]

{'type': 'PvP',
 'battleTime': '20200409T221510.000Z',
 'isLadderTournament': False,
 'arena': {'id': 54000042, 'name': 'Legendary Arena'},
 'gameMode': {'id': 72000006, 'name': 'Ladder'},
 'deckSelection': 'collection',
 'team': [{'tag': '#2CC8VPCP',
   'name': 'mike1m1ke',
   'startingTrophies': 6490,
   'trophyChange': 20,
   'crowns': 1,
   'kingTowerHitPoints': 5832,
   'princessTowersHitPoints': [3668, 140],
   'clan': {'tag': '#Y0CV8L0J', 'name': 'YouTube', 'badgeId': 16000164},
   'cards': [{'name': 'X-Bow',
     'id': 27000008,
     'level': 8,
     'starLevel': 3,
     'maxLevel': 8,
     'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/zVQ9Hme1hlj9Dc6e1ORl9xWwglcSrP7ejow5mAhLUJc.png'}},
    {'name': 'Tesla',
     'id': 27000006,
     'level': 13,
     'starLevel': 2,
     'maxLevel': 13,
     'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/OiwnGrxFMNiHetYEerE-UZt0L_uYNzFY7qV_CA_OxR4.png'}},
    {'name': 'The Log',
     'id': 28000011,

In [ ]:
locations = get_list_locations()

In [ ]:
location_ranking_players = get_list_locations_rankings_players(57000249)

In [ ]:
top_ranking_player_us = get_top_ranking_player_by_location(57000249)

In [ ]:
top_ranking_player_us

In [ ]:
get_player_info(top_ranking_player_us)

In [ ]:
locations['items']

In [ ]:
df_locations = pd.DataFrame(locations['items'])
df_locations

In [ ]:
df_locations.loc[df_locations['name']=='United States']

In [ ]:
# get_player_info(player_tag)

In [ ]:
player_battlelog = get_player_battlelog(player_tag)

In [ ]:
len(player_battlelog)

In [ ]:
for battle in player_battlelog:
    if battle['type'] == 'clanWarWarDay':
        print(battle)

In [ ]:
df = pd.DataFrame(json_response['items'], index=df['index'])
df = df.drop(['index'], axis=1)
df

In [172]:
kenny = '#9UPGRYYQ'
julio = '#C2L8C8JL'

In [175]:
card_info = get_player_info(kenny)
df = pd.DataFrame(card_info['cards'])

card_rarity_common = (df['maxLevel'] == 13)
card_rarity_rare =  (df['maxLevel'] == 11)
card_rarity_epic = (df['maxLevel'] == 8)
card_rarity_leg = (df['maxLevel'] == 5)

card_rarity_conditions = [card_rarity_common, card_rarity_rare, card_rarity_epic, card_rarity_leg]
card_rarity_types = ['Common', 'Rare', 'Epic', 'Legendary']
card_rarity_rate = [1, 2, 3, 4]
card_required_maxLevel = [9586, 2586, 386, 36]

df['card_rarity'] = np.select(card_rarity_conditions, card_rarity_types, default="n/a")
df['card_rarity_rate'] = np.select(card_rarity_conditions, card_rarity_rate, default=0)
df['maxCard']     = np.select(card_rarity_conditions, card_required_maxLevel, default=0)

cols = list(df.columns.values)
cols = ['id', 'name', 'level', 'maxLevel', 'card_rarity', 'card_rarity_rate', 'count', 'iconUrls', 'starLevel', 'maxCard']
df = df[cols]

df.drop(columns=['iconUrls', 'starLevel'], inplace=True)

df = df.set_index('id')

In [176]:
card_rarity_common_lvl_1  = (df['maxLevel'] == 13) & (df['level'] == 1)
card_rarity_common_lvl_2  = (df['maxLevel'] == 13) & (df['level'] == 2)
card_rarity_common_lvl_3  = (df['maxLevel'] == 13) & (df['level'] == 3)
card_rarity_common_lvl_4  = (df['maxLevel'] == 13) & (df['level'] == 4)
card_rarity_common_lvl_5  = (df['maxLevel'] == 13) & (df['level'] == 5)
card_rarity_common_lvl_6  = (df['maxLevel'] == 13) & (df['level'] == 6)
card_rarity_common_lvl_7  = (df['maxLevel'] == 13) & (df['level'] == 7)
card_rarity_common_lvl_8  = (df['maxLevel'] == 13) & (df['level'] == 8)
card_rarity_common_lvl_9  = (df['maxLevel'] == 13) & (df['level'] == 9)
card_rarity_common_lvl_10 = (df['maxLevel'] == 13) & (df['level'] == 10)
card_rarity_common_lvl_11 = (df['maxLevel'] == 13) & (df['level'] == 11)
card_rarity_common_lvl_12 = (df['maxLevel'] == 13) & (df['level'] == 12)
card_rarity_common_lvl_13 = (df['maxLevel'] == 13) & (df['level'] == 13)

card_rarity_rare_lvl_1  = (df['maxLevel'] == 11) & (df['level'] == 1)
card_rarity_rare_lvl_2  = (df['maxLevel'] == 11) & (df['level'] == 2)
card_rarity_rare_lvl_3  = (df['maxLevel'] == 11) & (df['level'] == 3)
card_rarity_rare_lvl_4  = (df['maxLevel'] == 11) & (df['level'] == 4)
card_rarity_rare_lvl_5  = (df['maxLevel'] == 11) & (df['level'] == 5)
card_rarity_rare_lvl_6  = (df['maxLevel'] == 11) & (df['level'] == 6)
card_rarity_rare_lvl_7  = (df['maxLevel'] == 11) & (df['level'] == 7)
card_rarity_rare_lvl_8  = (df['maxLevel'] == 11) & (df['level'] == 8)
card_rarity_rare_lvl_9  = (df['maxLevel'] == 11) & (df['level'] == 9)
card_rarity_rare_lvl_10 = (df['maxLevel'] == 11) & (df['level'] == 10)
card_rarity_rare_lvl_11 = (df['maxLevel'] == 11) & (df['level'] == 11)

card_rarity_epic_lvl_1  = (df['maxLevel'] == 8) & (df['level'] == 1)
card_rarity_epic_lvl_2  = (df['maxLevel'] == 8) & (df['level'] == 2)
card_rarity_epic_lvl_3  = (df['maxLevel'] == 8) & (df['level'] == 3)
card_rarity_epic_lvl_4  = (df['maxLevel'] == 8) & (df['level'] == 4)
card_rarity_epic_lvl_5  = (df['maxLevel'] == 8) & (df['level'] == 5)
card_rarity_epic_lvl_6  = (df['maxLevel'] == 8) & (df['level'] == 6)
card_rarity_epic_lvl_7  = (df['maxLevel'] == 8) & (df['level'] == 7)
card_rarity_epic_lvl_8  = (df['maxLevel'] == 8) & (df['level'] == 8)

card_rarity_leg_lvl_1  = (df['maxLevel'] == 5) & (df['level'] == 1)
card_rarity_leg_lvl_2  = (df['maxLevel'] == 5) & (df['level'] == 2)
card_rarity_leg_lvl_3  = (df['maxLevel'] == 5) & (df['level'] == 3)
card_rarity_leg_lvl_4  = (df['maxLevel'] == 5) & (df['level'] == 4)
card_rarity_leg_lvl_5  = (df['maxLevel'] == 5) & (df['level'] == 5)

card_lvl = [card_rarity_common_lvl_1, card_rarity_common_lvl_2, card_rarity_common_lvl_3, card_rarity_common_lvl_4, 
         card_rarity_common_lvl_5, card_rarity_common_lvl_6, card_rarity_common_lvl_7, card_rarity_common_lvl_8, 
         card_rarity_common_lvl_9, card_rarity_common_lvl_10, card_rarity_common_lvl_11, card_rarity_common_lvl_12, 
         card_rarity_common_lvl_13, card_rarity_rare_lvl_1, card_rarity_rare_lvl_2, card_rarity_rare_lvl_3, 
         card_rarity_rare_lvl_4, card_rarity_rare_lvl_5, card_rarity_rare_lvl_6, card_rarity_rare_lvl_7, 
         card_rarity_rare_lvl_8, card_rarity_rare_lvl_9, card_rarity_rare_lvl_10, card_rarity_rare_lvl_11, 
         card_rarity_epic_lvl_1, card_rarity_epic_lvl_2, card_rarity_epic_lvl_3, card_rarity_epic_lvl_4, 
         card_rarity_epic_lvl_5, card_rarity_epic_lvl_6, card_rarity_epic_lvl_7, card_rarity_epic_lvl_8, 
         card_rarity_leg_lvl_1, card_rarity_leg_lvl_2, card_rarity_leg_lvl_3, card_rarity_leg_lvl_4, 
         card_rarity_leg_lvl_5]

cards_req = [0, 2, 6, 16, 
             36, 86, 186, 386, 
             786, 1586, 2586, 4586, 
             9586, 0, 2, 6, 
             16, 36, 86, 186, 
             386, 786, 1586, 2586, 
             0, 2, 6, 16, 
             36, 86, 186, 386, 
             0, 2, 6, 16, 
             36]

df['total_cards']     = np.select(card_lvl, cards_req, default=0)
df['cards_needed_maxLevel'] = ( (df['maxCard']) - (df['count']+df['total_cards']) )

# df.sort_index()
df.sort_values(['card_rarity_rate', 'cards_needed_maxLevel'])

,name,level,maxLevel,card_rarity,card_rarity_rate,count,maxCard,total_cards,cards_needed_maxLevel
id,,,,,,,,,
28000001,Arrows,11,13,Common,1,7000,9586,2586,0
27000002,Mortar,9,13,Common,1,8800,9586,786,0
27000000,Cannon,10,13,Common,1,8000,9586,1586,0
26000010,Skeletons,11,13,Common,1,7000,9586,2586,0
26000024,Royal Giant,10,13,Common,1,8000,9586,1586,0
26000031,Fire Spirits,13,13,Common,1,0,9586,9586,0
26000030,Ice Spirit,11,13,Common,1,7000,9586,2586,0
26000019,Spear Goblins,11,13,Common,1,7000,9586,2586,0
28000008,Zap,13,13,Common,1,0,9586,9586,0
